In [47]:
import wrangle as w 
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
from time import time

import pre_processing_nick as pp

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from matplotlib.ticker import ScalarFormatter
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector

In [2]:
df = pp.create_modeling_df()

Using cached CSV
the shape of this modeling df should be (6893, 46)
the current shape is (6893, 46)
please split and then scale this dataframe


In [3]:
X_train, X_validate, X_test,\
y_train, y_validate, y_test = pp.split_and_scale(df)

In [4]:
print('X_train:', X_train.shape)
print('X_validate:', X_validate.shape)
print('X_test:', X_test.shape)

X_train: (3859, 46)
X_validate: (1655, 46)
X_test: (1379, 46)


## Feature Selection
Trying a new approach to feature selection using one of Scikitlearn's transformers. 
This Sequential Feature Selector adds (forward selection) or removes (backward selection) features to form a feature subset in a greedy fashion. At each stage, this estimator chooses the best feature to add or remove based on the cross-validation score of an estimator.

In [49]:
forest = RandomForestClassifier(n_estimators=8,
                               criterion='entropy')
sfs = SequentialFeatureSelector(forest, n_features_to_select=8)
sfs1 = sfs.fit(X_train, y_train)



In [39]:
sfs.get_support()

array([ True, False,  True,  True,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False])

In [ ]:
# which gives the following best features.
sfs_features = ['budget', 'vote_average', 'vote_count', 'release_year',
                'is_genre_comedy', 'cast_actor_1_Robin Williams',
                'production_company_TwentiethCenturyFoxFilmCorporation',
                'production_company_other_company']    